Machine learning project - Rémi Ollivier & Vincent Durpoix

Utilisation de la donnée de speed dating 

In [24]:
#Import database
import pandas as pd

df=pd.read_csv("SpeedDatingData.csv",encoding = "ISO-8859-1")


In [25]:
#Remove irrelevent data
df = df.drop("iid", axis=1)
df = df.drop("id", axis=1)
df = df.drop("partner", axis=1)




Essaie 1 : en enlevant toutes les columns ou les data etaient nuls -- > 12 columns pas forcément toutes usefull
Monter que c'étais pas préci


Essaie 2 : Replace NaN by 0 and remove useless columns manually
Plus précis --> affiner les resultats en essayant de changer des variables
    

Essaie 3 : Essayer avec réseau de neurone


In [26]:
df1 = df

#Maybe usefull later mn_sat
df1 = df1.drop("like", axis=1)
df1 = df1.drop("like_o", axis=1)
df1 = df1.drop("field", axis=1)
df1 = df1.drop("mn_sat", axis=1)
df1 = df1.drop("tuition", axis=1)
df1 = df1.drop("from", axis=1)
df1 = df1.drop("career", axis=1)
df1 = df1.drop("zipcode", axis=1)
df1 = df1.drop("income", axis=1)
df1 = df1.drop("undergra", axis=1)
df1 = df1.drop("dec", axis=1)
df1 = df1.drop("dec_o", axis=1)


#Remove where columns value = nan
#df1 = df1.dropna(axis=1, how='any')
#or replace Nan by 0 
df1 = df1.fillna(0)



#Some Stats
#Total
print(df1.shape)

#Repartition home/femme
men = df1.loc[df1['gender'] == 1]
women = df1.loc[df1['gender'] == 0]
print("Number of men : ", men.shape[0])
print("Number of women : ",women.shape[0])

#Women/men matchin
print("Men who match :",men.loc[df1['match'] == 1].shape[0])
print("Women who match :",women.loc[df1['match'] == 0].shape[0])



(8378, 180)
Number of men :  4194
Number of women :  4184
Men who match : 690
Women who match : 3494


In [28]:
#Classifieur arbre
from sklearn.tree import DecisionTreeClassifier
import random
from sklearn.datasets import load_iris
iris = load_iris()

def accuracy(y_pred,y) : 
    return (sum(y_pred == y))/len(y)

def data_split(x, y, prob):
        train,test=[],[]
        d=zip(x,y)
        for line in d:
            if(random.random()<prob):
                train.append(line)
            else :
                test.append(line)
        
        x_train,y_train=list(zip(*train))
        x_test,y_test=list(zip(*test))
        return x_train,y_train, x_test,y_test

#x=iris.data
#y=iris.target        

#print(x)
#print(y)

#x_train,y_train, x_test,y_test=data_split(x, y, .5)
#print(y_train,x_train)
#clf = DecisionTreeClassifier().fit(x_train,y_train)
#y_pred=clf.predict(x_test)
#print(accuracy(y_pred,y_test))

y = df1["match"].as_matrix()
x = df1.drop("match", axis=1).as_matrix()


x_train,y_train, x_test,y_test=data_split(x, y, .5)
clf = DecisionTreeClassifier().fit(x_train,y_train)
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(x_train,y_train)

y_pred=clf.predict(x_test)

print(accuracy(y_pred,y_test))

print(clf)



0.790664443915
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [ ]:
from sklearn.tree import export_graphviz
from IPython.core.display import Image, display
with open("tree.dot", 'w') as f:
        export_graphviz(clf, out_file=f,
                        feature_names=df1.drop("match", axis=1).columns.values,
                        filled=True)

In [ ]:
#Visualiser l'arbre
!dot -Tpng tree.dot -o tree.png 

In [ ]:
display(Image('tree.png'))